In [2]:
import os
import requests

pdf_path="C:\Study\Programming\PYTHON\Learn_Pytorch\Data\Human-Nutrition-2020-Edition-1598491699.pdf"

if not os.path.exists(pdf_path):
    print(f"[INFO] file doesn't Exist")
    
    url='https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf'

    filename= pdf_path

    response=requests.get(url)

    if response.status_code == 200:
        with open(filename,"wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been downloaded and save as {filename}")
    else:
        print(f"[INFO] failed to download the file. Status code: {response.status_code}")

else:
    print(f"File {pdf_path} Exist")

File C:\Study\Programming\PYTHON\Learn_Pytorch\Data\Human-Nutrition-2020-Edition-1598491699.pdf Exist


<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\MIT\AppData\Local\Temp\ipykernel_3120\2279267867.py:4: SyntaxWarning: invalid escape sequence '\S'
  pdf_path="C:\Study\Programming\PYTHON\Learn_Pytorch\Data\Human-Nutrition-2020-Edition-1598491699.pdf"


In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter (text:str) -> str:
    cleaned_text=text.replace("\n"," ").strip()

    return cleaned_text

def open_and_read_pdf(pdf_path:str) ->list[dict]:
    doc=fitz.open(pdf_path)
    pages_and_texts=[]
    for page_number, page in tqdm(enumerate(doc)):
        text= page.get_text()
        text=text_formatter(text=text)
        pages_and_texts.append({"Page_number": page_number - 41,
                                "Page_char_count": len(text),
                                "Page_word_count": len(text.split(" ")),
                                "Page_sentences_count": len(text.split(". ")),
                                "Page_token_count": len(text)/4,
                                "text":text
                                })
    return pages_and_texts

pages_and_text= open_and_read_pdf(pdf_path=pdf_path)
pages_and_text[:2]

c:\Study\Programming\PYTHON\Learn_Pytorch\learn_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1208it [00:01, 1087.34it/s]


[{'Page_number': -41,
  'Page_char_count': 29,
  'Page_word_count': 4,
  'Page_sentences_count': 1,
  'Page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'Page_number': -40,
  'Page_char_count': 0,
  'Page_word_count': 1,
  'Page_sentences_count': 1,
  'Page_token_count': 0.0,
  'text': ''}]

In [4]:
import random

random.sample(pages_and_text, k=2)

[{'Page_number': 655,
  'Page_char_count': 1121,
  'Page_word_count': 198,
  'Page_sentences_count': 11,
  'Page_token_count': 280.25,
  'text': 'Image by  Allison  Calabrese /  CC BY 4.0  Iron  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Red blood cells contain the oxygen-carrier protein hemoglobin. It  is composed of four globular peptides, each containing a heme  complex. In the center of each heme, lies iron (Figure 11.2). Iron is  needed for the production of other iron-containing proteins such as  myoglobin. \xa0Myoglobin is a protein found in the muscle tissues that  enhances the amount of available oxygen for muscle contraction.  Iron is also a key component of hundreds of metabolic enzymes.  Many of the proteins of the electron-transport chain contain  iron–sulfur clusters involved in the transfer of high-energy  electrons and ultimately ATP synthesis. Iron is also involved in  numerous metabolic reactions that take pla

In [5]:
import pandas as pd

df=pd.DataFrame(pages_and_text)
df.iloc[55]

Page_number                                                            14
Page_char_count                                                       947
Page_word_count                                                       167
Page_sentences_count                                                    6
Page_token_count                                                   236.75
text                    Image by  David De  Veroli on  unsplash.co m /...
Name: 55, dtype: object

In [6]:
df.describe()

,Page_number,Page_char_count,Page_word_count,Page_sentences_count,Page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.004139,198.299669,9.972682,287.001035
std,348.86387,560.382275,95.759336,6.187226,140.095569
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.000000,134.000000,4.000000,190.500000
50%,562.50000,1231.500000,214.500000,10.000000,307.875000
75%,864.25000,1603.500000,271.000000,14.000000,400.875000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


### NLP Library Spacy

In [7]:
from spacy.lang.en import English
nlp=English()

nlp.add_pipe("sentencizer")

doc= nlp("First Sentence. Sencond sentence. Third sentence.")
assert len(list(doc.sents))==3

list(doc.sents)

[First Sentence., Sencond sentence., Third sentence.]

In [8]:
pages_and_text[80]

{'Page_number': 39,
 'Page_char_count': 1166,
 'Page_word_count': 201,
 'Page_sentences_count': 7,
 'Page_token_count': 291.5,
 'text': 'departments of the US federal government. The USDA and HHS  websites are great tools for discovering ways to optimize health;  however, it is important to gather nutrition information from  multiple resources, as there are often differences in opinion among  various scientists and public health organizations. Full text versions  of  the  DRI  publications  are  available  in  pdf  format  at  https://www.nap.edu/, along with many other free publications.  Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mobile  devices, however, for

In [9]:
for items in tqdm(pages_and_text):
    items["sentences"]=list(nlp(items["text"]).sents)

    items["sentences"] = [str(sentence) for  sentence in items["sentences"]]

    items["page_sentence_count_spacy"] = len(items["sentences"])

100%|██████████| 1208/1208 [00:01<00:00, 770.57it/s]


In [10]:
random.sample(pages_and_text, k=1)

[{'Page_number': 559,
  'Page_char_count': 863,
  'Page_word_count': 136,
  'Page_sentences_count': 8,
  'Page_token_count': 215.75,
  'text': 'Image by  Allison  Calabrese /  CC BY 4.0  Korsakoff syndrome can cause similar symptoms as beriberi such  as confusion, loss of coordination, vision changes, hallucinations,  and may progress to coma and death. This condition is specific  to alcoholics as diets high in alcohol can cause thiamin deficiency.  Other individuals at risk include individuals who also consume diets  typically low in micronutrients such as those with eating disorders,  elderly, and individuals who have gone through gastric bypass  surgery.5  Figure 9.10 The Role of Thiamin  Figure 9.11 Beriberi, Thiamin Deficiency  5.\xa0Fact Sheets for Health Professionals: Thiamin. National  Institute of Health, Office of Dietary Supplements.  \xa0https://ods.od.nih.gov/factsheets/Thiamin- HealthProfessional/. Updated Feburary 11, 2016.  Accessed October 22, 2017.  Water-Soluble Vit

In [12]:
df= pd.DataFrame(pages_and_text)
df.describe().round(2)

,Page_number,Page_char_count,Page_word_count,Page_sentences_count,Page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00,10.32
std,348.86,560.38,95.76,6.19,140.10,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.00,134.00,4.00,190.50,5.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00
75%,864.25,1603.50,271.00,14.00,400.88,15.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00


In [15]:
num_sentence_chunk_size=10

def split_list(input_list:list[str],
               slice_size:int=num_sentence_chunk_size)->list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)] 

test_list= list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [17]:
for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"]=len(item["sentence_chunks"])


100%|██████████| 1208/1208 [00:00<00:00, 409895.58it/s]


In [18]:
random.sample(pages_and_text, k=1)

[{'Page_number': 748,
  'Page_char_count': 1704,
  'Page_word_count': 288,
  'Page_sentences_count': 15,
  'Page_token_count': 426.0,
  'text': 'Nutrition Facts label provides the information to help you make  healthful decisions.  In short, substituting vegetables and fruits in place of foods high  in added sugars, saturated fats, and sodium is a good way to make  a nutrient-poor diet healthy again. Vegetables are full of nutrients  and antioxidants that help promote good health and reduce the  risk for developing chronic diseases such as stroke, heart disease,  high blood pressure, Type 2 diabetes, and certain types of cancer.  Starting with these small shifts in your diet as mentioned above will  boost your overall health profile.  Discretionary Calories  When following a balanced, healthful diet with many nutrient-dense  foods, you may consume enough of your daily nutrients before  you reach your daily calorie limit. The remaining calories are  discretionary (to be used according t

In [19]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,Page_number,Page_char_count,Page_word_count,Page_sentences_count,Page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00,10.32,1.53
std,348.86,560.38,95.76,6.19,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,4.00,190.50,5.00,1.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00,1.00
75%,864.25,1603.50,271.00,14.00,400.88,15.00,2.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00,3.00


In [24]:
import re

pages_and_chunks=[]
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict={}
        chunk_dict["page_number"]=item["Page_number"]

        joined_sentence_chunk = "".join(sentence_chunk).replace(" "," ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])',r'. \1', joined_sentence_chunk)

        chunk_dict["sentence_chunk"]= joined_sentence_chunk

        chunk_dict["chunk_char_count"]= len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"]= len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"]= len(joined_sentence_chunk)/4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

100%|██████████| 1208/1208 [00:00<00:00, 42422.40it/s]


1843

In [25]:
random.sample(pages_and_chunks,k=1)

[{'page_number': 709,
  'sentence_chunk': 'Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document). Learning activities may be used across various mobile  devices, however, for the best user experience it is strongly  recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome. \xa0 An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=409  Introduction  |  709',
  'chunk_char_count': 741,
  'chunk_word_count': 117,
  'chunk_token_count': 185.25}]

In [29]:
df=pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,750.01,127.90,187.50
std,347.79,455.70,79.99,113.93
min,-41.00,14.00,4.00,3.50
25%,280.50,321.50,51.00,80.38
50%,586.00,763.00,131.00,190.75
75%,890.00,1137.50,194.00,284.38
max,1166.00,1870.00,412.00,467.50


In [30]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f"Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}")

Chunk token count: 7.5 | Text: Water-Soluble Vitamins  |  553
Chunk token count: 6.25 | Text: You can  Chloride  |  193
Chunk token count: 18.0 | Text: Published August 2011. Accessed September 22, 2017. Introduction  |  147
Chunk token count: 10.25 | Text: 920  |  Older Adulthood: The Golden Years
Chunk token count: 22.75 | Text: Figure 6.10 Enzymes Role in Carbohydrate Digestion  Protein’s Functions in the Body  |  385


In [31]:
pages_and_chunks_over_min_token_len=df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE',
  'chunk_char_count': 320,
  'chunk_word_count': 54,
  'chunk_token_count': 80.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and  Human Nutrition Program is licensed under a Creative Commons Attribution 4.0  International License, except where otherwise noted.',
  'chunk_char_count': 212,
  'chunk_word_count': 32,
  'chunk_token_count': 53.0}]

In [32]:
random.sample(pages_and_chunks_over_min_token_len, k =1)

[{'page_number': 781,
  'sentence_chunk': 'The period from birth to eighteen. • Onset of Puberty. Can occur from ages nine to fourteen, which  is the beginning of adolescence. • Older adolescence. The stage that takes place between ages  fourteen and eighteen. • Adulthood. The period from the end of adolescence to the end  of life and begins at age eighteen in the US and nineteen in  some other countries. • Middle age. The period of adulthood that stretches from age  forty-five to sixty-four. • Senior years, or old age.',
  'chunk_char_count': 482,
  'chunk_word_count': 89,
  'chunk_token_count': 120.5}]